In [2]:
import os
import glob
import pandas as pd

In [3]:
basepath = "/home/abetatos/Escritorio/TFM_models/"
FEATURES = ["DEM", "SCE", "Aspect", "Slope", "TPI", "TPIWGW", "DEMSNOW", "FFSNOW"]

In [4]:
models = ["withoutmask_all_sgd", "withoutmask_all_sgd_adagrad", "withoutmask_all_sgd_adam"]

all_df = pd.DataFrame()
for model in models:
    files = glob.glob(basepath + model + "/*.pt")
    files_df = pd.DataFrame(files, columns=['filepath'])
    files_df['modeltype'] = model
    files_df['model'] = files_df['filepath'].apply(lambda x: os.path.basename(x).split("_")[0])
    files_df['variable'] = files_df['filepath'].apply(lambda x: x.split("_")[-1].replace(".pt", ""))
    files_df['report'] = files_df['model'].apply(lambda x: os.path.join(basepath, model, f"{x}_train_report.csv"))
    files_df['best'] = files_df['model'].apply(lambda x: os.path.join(basepath, model, f"{x}_best_report.csv"))
    # files_df.set_index('variable', inplace=True)

    all_df = pd.concat([all_df, files_df])


all_df['number_length'] = all_df['variable'].astype(str).apply(len)
all_df = all_df.sort_values(by='number_length')
all_df = all_df.drop(columns='number_length')
all_df


,filepath,modeltype,model,variable,report,best
0,/home/abetatos/Escritorio/TFM_models/withoutma...,withoutmask_all_sgd,1685489702,01234567,/home/abetatos/Escritorio/TFM_models/withoutma...,/home/abetatos/Escritorio/TFM_models/withoutma...
0,/home/abetatos/Escritorio/TFM_models/withoutma...,withoutmask_all_sgd_adagrad,1685520448,01234567,/home/abetatos/Escritorio/TFM_models/withoutma...,/home/abetatos/Escritorio/TFM_models/withoutma...
0,/home/abetatos/Escritorio/TFM_models/withoutma...,withoutmask_all_sgd_adam,1685578066,01234567,/home/abetatos/Escritorio/TFM_models/withoutma...,/home/abetatos/Escritorio/TFM_models/withoutma...
1,/home/abetatos/Escritorio/TFM_models/withoutma...,withoutmask_all_sgd_adam,1685536886,01234567,/home/abetatos/Escritorio/TFM_models/withoutma...,/home/abetatos/Escritorio/TFM_models/withoutma...
2,/home/abetatos/Escritorio/TFM_models/withoutma...,withoutmask_all_sgd_adam,1685534484,01234567,/home/abetatos/Escritorio/TFM_models/withoutma...,/home/abetatos/Escritorio/TFM_models/withoutma...


In [5]:
best_models = []
for variable, var_df in all_df.groupby("variable"):
    print(variable)
    for index, series in var_df.iterrows():
        train_report = pd.read_csv(series['best'])
        train_diff = train_report['Val loss 2'].values[0].round(3)
        
        # best_models.append([series['modeltype'], train_diff, FEATURES[int(variable)]])
        # best_models.append([series['modeltype'], train_diff, variable])
        # best_models.append([series['modeltype'], train_diff, [FEATURES[v] for v in range(8) if str(v) not in variable]]) #Leave one out
        best_models.append([series['modeltype'], train_diff, [FEATURES[int(v)] for v in variable]]) #Iterative

best_models_df = pd.DataFrame(best_models, columns=["modelname", "score", "variable"])
best_by_model_df = pd.DataFrame()
for model, aux_df in best_models_df.groupby("modelname"):
    for index, series in aux_df.iterrows():
        best_by_model_df.at[model, str(series['variable'])] = series['score']

best_by_model_df.style.highlight_min(color = 'blue', 
                       axis = 0)

01234567


,"['DEM', 'SCE', 'Aspect', 'Slope', 'TPI', 'TPIWGW', 'DEMSNOW', 'FFSNOW']"
withoutmask_all_sgd,0.564000
withoutmask_all_sgd_adagrad,0.565000
withoutmask_all_sgd_adam,0.550000


In [23]:
best_by_model_df.transpose().sort_values(by="iterative_sgd_adam")

,iterative_sgd_adam
"['DEM', 'SCE', 'Aspect', 'Slope', 'TPI', 'FFSNOW']",0.524
"['DEM', 'SCE', 'Aspect', 'Slope', 'TPI', 'DEMSNOW', 'FFSNOW']",0.529
"['DEM', 'SCE', 'Slope', 'TPI']",0.530
"['DEM', 'SCE', 'Aspect', 'Slope', 'TPI', 'TPIWGW', 'DEMSNOW', 'FFSNOW']",0.531
"['DEM', 'SCE', 'Aspect', 'Slope', 'TPI']",0.532
"['DEM', 'SCE', 'Slope']",0.548
"['DEM', 'Slope']",0.576
['DEM'],0.607
